In [2]:
import requests
from bs4 import BeautifulSoup
import json
from pandas import DataFrame as df
import pandas as pd

page = requests.get("https://www.walmart.com/store/directory")
soup = BeautifulSoup(page.content,"html.parser")

states_links = []
cities_links = []
store_info = []

states = soup.select('.list-ul li a')
for i in states:
    states_links.append("https://www.walmart.com" + i['href'])

for statelink in states_links:
    state_page = requests.get(statelink)
    state_soup = BeautifulSoup(state_page.content,"html.parser")
    
    cities = state_soup.select('.list-ul li a')
    for city in cities:
        cities_links.append("https://www.walmart.com" + city['href'])    
    
for citylink in cities_links:
    city_page = requests.get(citylink)
    city_soup = BeautifulSoup(city_page.content,"html.parser") 

    try:
        addressline1 = city_soup.find(class_= 'store-address-line-1',text = True)    
        addressline1_text = addressline1.get_text()

        addressline2 = city_soup.find(class_ = "store-address-city",text = True)
        addressline2_text = addressline2.get_text()

        storestate = city_soup.find(class_ = "store-address-state",text = True)
        state_text = storestate.get_text()

        postalcode = city_soup.find(class_ = "store-address-postal",text = True)
        postalcode_text = postalcode.get_text()

        data = {"streetAddress" : addressline1_text,
             "addressLocality" : addressline2_text,
             "addressRegion" : state_text,
             "postalCode" : postalcode_text,
             "link" : citylink,
             "store" : "Walmart"}

        store_info.append(data)
    except:
        pass

stores_df = df.from_records(store_info)
stores_df.to_csv("Walmart_StateList.csv",index = False)